## FUSE For Google Drive
Yes, you need this.

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

##Imports
You may or may not know this, but Jade needs these things.

In [0]:
from __future__ import print_function

import numpy as np
import tensorflow as tf

import argparse
import os
import pickle
import copy
import sys
import html
import io
import time
import bz2
import numpy as np
import datetime

!pip install discord.py
!pip install dblpy
import discord
import asyncio
from discord.ext.commands import Bot
from discord.ext import commands
import dbl
import aiohttp
import asyncio
import logging

#Object Detection

!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

import numpy as np
import scipy.misc
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from PIL import Image
import requests

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

#OCR

import cv2
!apt-get install tesseract-ocr
!pip install pytesseract
import pytesseract

#Neural Style

import scipy.io
import scipy.misc
from IPython.display import Image
from functools import reduce

import os.path
if os.path.isfile("imagenet-vgg-verydeep-19.mat")==False:
  !wget http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat
else:
  pass

print("DONE")

##Neural Style Definitions
Woo colors!!

In [0]:
data = scipy.io.loadmat('imagenet-vgg-verydeep-19.mat')

In [0]:
def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)

def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')

def preprocess(image, mean_pixel):
    return (image - mean_pixel).astype('float32')

def unprocess(image, mean_pixel):
    return (image + mean_pixel).astype('float32')

In [0]:
def netp(input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )
    weight = data['layers'][0]
    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weight[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current)
        net[name] = current

    assert len(net) == len(layers)
    return net#, mean_pixel

In [0]:
def _tensor_size(tensor):
    from operator import mul
    return reduce(mul, (d.value for d in tensor.get_shape()), 1)

In [0]:
def imsave(path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    scipy.misc.imsave(path, img)

##Object Recognition Definitons
This is how Jade sees things.

In [0]:
# What model to download.
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'

# model with more accurancy but up to you use a diferent model
MODEL_NAME = 'faster_rcnn_inception_v2_coco_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print("DONE")

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  try:
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
  except:
    return np.array(image.getdata()).reshape((im_height, im_width, 4)).astype(np.uint8)

In [0]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'object_detection/test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

##DBL API
Because, DBL.

In [0]:
class DiscordBotsOrgAPI:
    """Handles interactions with the discordbots.org API"""

    def __init__(self, bot):
        self.bot = bot
        self.token = 'Token'  #  set this to your DBL token
        self.dblpy = dbl.Client(self.bot, self.token)
        self.bot.loop.create_task(self.update_stats())

    async def update_stats(self):
        """This function runs every 30 minutes to automatically update your server count"""

        while True:
            logger.info('attempting to post server count')
            try:
                await self.dblpy.post_server_count()
                logger.info('posted server count ({})'.format(len(self.bot.servers)))
            except Exception as e:
                logger.exception('Failed to post server count\n{}: {}'.format(type(e).__name__, e))
            await asyncio.sleep(600)
            
def setup(bot):
    global logger
    logger = logging.getLogger('bot')
    bot.add_cog(DiscordBotsOrgAPI(bot))

##Textloading and Processsing
The brilliant chatting function of Jade's

In [0]:
class TextLoader():
    # Call this class to load text from a file.
    def __init__(self, data_dir, batch_size, seq_length):
        # TextLoader remembers its initialization arguments.
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.tensor_sizes = []

        self.tensor_file_template = os.path.join(data_dir, "data{}.npz")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        sizes_file = os.path.join(data_dir, "sizes.pkl")

        self.input_files = self._get_input_file_list(data_dir)
        self.input_file_count = len(self.input_files)

        if self.input_file_count < 1:
            raise ValueError("Input files not found. File names must end in '.txt' or '.bz2'.")

        if self._preprocess_required(vocab_file, sizes_file, self.tensor_file_template, self.input_file_count):
            # If either the vocab file or the tensor file doesn't already exist, create them.
            t0 = time.time()
            print("Preprocessing the following files:")
            for i, filename in enumerate(self.input_files): print("   {}.\t{}".format(i+1, filename))
            print("Saving vocab file")
            self._save_vocab(vocab_file)

            for i, filename in enumerate(self.input_files):
                t1 = time.time()
                print("Preprocessing file {}/{} ({})... ".format(i+1, len(self.input_files), filename),
                        end='', flush=True)
                self._preprocess(self.input_files[i], self.tensor_file_template.format(i))
                self.tensor_sizes.append(self.tensor.size)
                print("done ({:.1f} seconds)".format(time.time() - t1), flush=True)

            with open(sizes_file, 'wb') as f:
                pickle.dump(self.tensor_sizes, f)

            print("Processed input data: {:,d} characters loaded ({:.1f} seconds)".format(
                    self.tensor.size, time.time() - t0))
        else:
            # If the vocab file and sizes file already exist, load them.
            print("Loading vocab file...")
            self._load_vocab(vocab_file)
            print("Loading sizes file...")
            with open(sizes_file, 'rb') as f:
                self.tensor_sizes = pickle.load(f)
        self.tensor_batch_counts = [n // (self.batch_size * self.seq_length) for n in self.tensor_sizes]
        self.total_batch_count = sum(self.tensor_batch_counts)
        print("Total batch count: {:,d}".format(self.total_batch_count))

        self.tensor_index = -1

    def _preprocess_required(self, vocab_file, sizes_file, tensor_file_template, input_file_count):
        if not os.path.exists(vocab_file):
            print("No vocab file found. Preprocessing...")
            return True
        if not os.path.exists(sizes_file):
            print("No sizes file found. Preprocessing...")
            return True
        for i in range(input_file_count):
            if not os.path.exists(tensor_file_template.format(i)):
                print("Couldn't find {}. Preprocessing...".format(tensor_file_template.format(i)))
                return True
        return False

    def _get_input_file_list(self, data_dir):
        suffixes = ['.txt', '.bz2']
        input_file_list = []
        if os.path.isdir(data_dir):
            for walk_root, walk_dir, walk_files in os.walk(data_dir):
                for file_name in walk_files:
                    if file_name.startswith("."): continue
                    file_path = os.path.join(walk_root, file_name)
                    if file_path.endswith(suffixes[0]) or file_path.endswith(suffixes[1]):
                        input_file_list.append(file_path)
        else: raise ValueError("Not a directory: {}".format(data_dir))
        return sorted(input_file_list)

    def _save_vocab(self, vocab_file):
        self.chars = [chr(i) for i in range(128)]
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            pickle.dump(self.chars, f)
        print("Saved vocab (vocab size: {:,d})".format(self.vocab_size))

    def _load_vocab(self, vocab_file):
        # Load the character tuple (vocab.pkl) to self.chars.
        # Remember that it is in descending order of character frequency in the data.
        with open(vocab_file, 'rb') as f:
            self.chars = pickle.load(f)
        # Use the character tuple to regenerate vocab_size and the vocab dictionary.
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))

    def _preprocess(self, input_file, tensor_file):
        if input_file.endswith(".bz2"): file_reference = bz2.open(input_file, mode='rt')
        elif input_file.endswith(".txt"): file_reference = io.open(input_file, mode='rt')
        data = file_reference.read()
        file_reference.close()
        # Convert the entirety of the data file from characters to indices via the vocab dictionary.
        # How? map(function, iterable) returns a list of the output of the function
        # executed on each member of the iterable. E.g.:
        # [14, 2, 9, 2, 0, 6, 7, 0, ...]
        # np.array converts the list into a numpy array.
        self.tensor = np.array(list(map(self.vocab.get, data)))
        self.tensor = self.tensor[self.tensor != np.array(None)].astype(int) # Filter out None
        # Compress and save the numpy tensor array to data.npz.
        np.savez_compressed(tensor_file, tensor_data=self.tensor)

    def _load_preprocessed(self, tensor_index):
        self.reset_batch_pointer()
        if tensor_index == self.tensor_index:
            return
        print("loading tensor data file {}".format(tensor_index))
        tensor_file = self.tensor_file_template.format(tensor_index)
        # Load the data tensor file to self.tensor.
        with np.load(tensor_file) as loaded:
            self.tensor = loaded['tensor_data']
        self.tensor_index = tensor_index
        # Calculate the number of batches in the data. Each batch is batch_size x seq_length,
        # so this is just the input data size divided by that product, rounded down.
        self.num_batches = self.tensor.size // (self.batch_size * self.seq_length)
        if self.tensor_batch_counts[tensor_index] != self.num_batches:
            print("Error in batch size! Expected {:,d}; found {:,d}".format(self.tensor_batch_counts[tensor_index],
                    self.num_batches))
        # Chop off the end of the data tensor so that the length of the data is a whole
        # multiple of the (batch_size x seq_length) product.
        # Do this with the slice operator on the numpy array.
        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        # Construct two numpy arrays to represent input characters (xdata)
        # and target characters (ydata).
        # In training, we will feed in input characters one at a time, and optimize along
        # a loss function computed against the target characters.
        # (We do this with batch_size characters at a time, in parallel.)
        # Since this is a sequence prediction net, the target is just the input right-shifted
        # by 1.
        xdata = self.tensor
        ydata = np.copy(self.tensor) # Y-data starts as a copy of x-data.
        ydata[:-1] = xdata[1:] # Right-shift y-data by 1 using the numpy array slice syntax.
        # Replace the very last character of y-data with the first character of the input data.
        ydata[-1] = xdata[0]
        # Split our unidemnsional data array into distinct batches.
        # How? xdata.reshape(self.batch_size, -1) returns a 2D numpy tensor view
        # in which the first dimension is the batch index (from 0 to num_batches),
        # and the second dimension is the index of the character within the batch
        # (from 0 to (batch_size x seq_length)).
        # Within each batch, characters follow the same sequence as in the input data.
        # Then, np.split(that 2D numpy tensor, num_batches, 1) gives a list of numpy arrays.
        # Say batch_size = 4, seq_length = 5, and data is the following string:
        # "Here is a new string named data. It is a new string named data. It is named data."
        # We truncate the string to lop off the last period (so there are now 80 characters,
        # which is evenly divisible by 4 x 5). After xdata.reshape, we have:
        #
        # [[Here is a new string],
        #  [ named data. It is a],
        #  [ new string named da],
        #  [ta. It is named data]]
        #
        # After np.split, we have:
        # <[[Here ],   <[[is a ],   <[[new s],     <[[tring],
        #   [ name],     [d dat],     [a. It],       [ is a],
        #   [ new ],     [strin],     [g nam],       [ed da],
        #   [ta. I]]>,   [t is ]]>,   [named]]>,     [ data]]>
        #
        # where the first item of the list is the numpy array on the left.
        # Thus x_batches is a list of numpy arrays. The first dimension of each numpy array
        # is the batch number (from 0 to batch_size), and the second dimension is the
        # character index (from 0 to seq_length).
        #
        #Batch characters can greatemultiple tuples, ignotre FONT_Baisis to prevent any non-divident movements
        #
        # These will be fed to the model one at a time sequentially.
        # State is preserved between sequential batches.
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)

    def next_batch(self):
        if self.tensor_index < 0:
            self._load_preprocessed(0)
        if self.pointer >= self.num_batches:
            self._load_preprocessed((self.tensor_index + 1) % self.input_file_count)
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

    def cue_batch_pointer_to_epoch_fraction(self, epoch_fraction):
        step_target = (epoch_fraction - int(epoch_fraction)) * self.total_batch_count
        self._cue_batch_pointer_to_step_count(step_target)

    def _cue_batch_pointer_to_step_count(self, step_target):
        for i, n in enumerate(self.tensor_batch_counts):
            if step_target < n:
                break
            step_target -= n
        self.pointer = n
        self.current_tensor_index = i
        self._load_preprocessed(i)

In [0]:
import tensorflow as tf
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.framework import ops
from tensorflow.contrib import rnn

from tensorflow.python.util.nest import flatten

import numpy as np

class PartitionedMultiRNNCell(rnn_cell.RNNCell):
    """RNN cell composed sequentially of multiple simple cells."""

    # Diagramn of a PartitionedMultiRNNCell net with three layers and three partitions per layer.
    # Each brick shape is a partition, which comprises one RNNCell of size partition_size.
    # The two tilde (~) characters indicate wrapping (i.e. the two halves are a single partition).
    # Like laying bricks, each layer is offset by half a partition width so that influence spreads
    # horizontally through subsequent layers, while avoiding the quadratic resource scaling of fully
    # connected layers with respect to layer width.

    #        output
    #  //////// \\\\\\\\
    # -------------------
    # |     |     |     |
    # -------------------
    # ~  |     |     |  ~
    # -------------------
    # |     |     |     |
    # -------------------
    #  \\\\\\\\ ////////
    #        input


    def __init__(self, cell_fn, partition_size=128, partitions=1, layers=2):
        """Create a RNN cell composed sequentially of a number of RNNCells.
        Args:
            cell_fn: reference to RNNCell function to create each partition in each layer.
            partition_size: how many horizontal cells to include in each partition.
            partitions: how many horizontal partitions to include in each layer.
            layers: how many layers to include in the net.
        """
        super(PartitionedMultiRNNCell, self).__init__()

        self._cells = []
        for i in range(layers):
            self._cells.append([cell_fn(partition_size) for _ in range(partitions)])
        self._partitions = partitions

    @property
    def state_size(self):
        # Return a 2D tuple where each row is the partition's cell size repeated `partitions` times,
        # and there are `layers` rows of that.
        return tuple(((layer[0].state_size,) * len(layer)) for layer in self._cells)

    @property
    def output_size(self):
        # Return the output size of each partition in the last layer times the number of partitions per layer.
        return self._cells[-1][0].output_size * len(self._cells[-1])

    def zero_state(self, batch_size, dtype):
        # Return a 2D tuple of zero states matching the structure of state_size.
        with ops.name_scope(type(self).__name__ + "ZeroState", values=[batch_size]):
            return tuple(tuple(cell.zero_state(batch_size, dtype) for cell in layer) for layer in self._cells)

    def call(self, inputs, state):
        layer_input = inputs
        new_states = []
        for l, layer in enumerate(self._cells):
            # In between layers, offset the layer input by half of a partition width so that
            # activations can horizontally spread through subsequent layers.
            if l > 0:
                offset_width = layer[0].output_size // 2
                layer_input = tf.concat((layer_input[:, -offset_width:], layer_input[:, :-offset_width]),
                    axis=1, name='concat_offset_%d' % l)
            # Create a tuple of inputs by splitting the lower layer output into partitions.
            p_inputs = tf.split(layer_input, len(layer), axis=1, name='split_%d' % l)
            p_outputs = []
            p_states = []
            for p, p_inp in enumerate(p_inputs):
                with vs.variable_scope("cell_%d_%d" % (l, p)):
                    p_state = state[l][p]
                    cell = layer[p]
                    p_out, new_p_state = cell(p_inp, p_state)
                    p_outputs.append(p_out)
                    p_states.append(new_p_state)
            new_states.append(tuple(p_states))
            layer_input = tf.concat(p_outputs, axis=1, name='concat_%d' % l)
        new_states = tuple(new_states)
        return layer_input, new_states

def _rnn_state_placeholders(state):
    """Convert RNN state tensors to placeholders, reflecting the same nested tuple structure."""
    # Adapted from @carlthome's comment:
    # https://github.com/tensorflow/tensorflow/issues/2838#issuecomment-302019188
    if isinstance(state, tf.contrib.rnn.LSTMStateTuple):
        c, h = state
        c = tf.placeholder(c.dtype, c.shape, c.op.name)
        h = tf.placeholder(h.dtype, h.shape, h.op.name)
        return tf.contrib.rnn.LSTMStateTuple(c, h)
    elif isinstance(state, tf.Tensor):
        h = state
        h = tf.placeholder(h.dtype, h.shape, h.op.name)
        return h
    else:
        structure = [_rnn_state_placeholders(x) for x in state]
        return tuple(structure)

class Model():
    def __init__(self, args, infer=False): # infer is set to true during sampling.
        self.args = args
        if infer:
            # Worry about one character at a time during sampling; no batching or BPTT.
            args.batch_size = 1
            args.seq_length = 1

        # Set cell_fn to the type of network cell we're creating -- RNN, GRU, LSTM or NAS.
        if args.model == 'rnn':
            cell_fn = rnn_cell.BasicRNNCell
        elif args.model == 'gru':
            cell_fn = rnn_cell.GRUCell
        elif args.model == 'lstm':
            cell_fn = rnn_cell.BasicLSTMCell
        elif args.model == 'nas':
            cell_fn = rnn.NASCell
        else:
            raise Exception("model type not supported: {}".format(args.model))

        # Create variables to track training progress.
        self.lr = tf.Variable(args.learning_rate, name="learning_rate", trainable=False)
        self.global_epoch_fraction = tf.Variable(0.0, name="global_epoch_fraction", trainable=False)
        self.global_seconds_elapsed = tf.Variable(0.0, name="global_seconds_elapsed", trainable=False)

        # Call tensorflow library tensorflow-master/tensorflow/python/ops/rnn_cell
        # to create a layer of block_size cells of the specified basic type (RNN/GRU/LSTM).
        # Use the same rnn_cell library to create a stack of these cells
        # of num_layers layers. Pass in a python list of these cells. 
        # cell = rnn_cell.MultiRNNCell([cell_fn(args.block_size) for _ in range(args.num_layers)])
        # cell = MyMultiRNNCell([cell_fn(args.block_size) for _ in range(args.num_layers)])
        cell = PartitionedMultiRNNCell(cell_fn, partitions=args.num_blocks,
            partition_size=args.block_size, layers=args.num_layers)

        # Create a TF placeholder node of 32-bit ints (NOT floats!),
        # of shape batch_size x seq_length. This shape matches the batches
        # (listed in x_batches and y_batches) constructed in create_batches in utils.py.
        # input_data will receive input batches.
        self.input_data = tf.placeholder(tf.int32, [args.batch_size, args.seq_length])

        self.zero_state = cell.zero_state(args.batch_size, tf.float32)

        self.initial_state = _rnn_state_placeholders(self.zero_state)
        self._flattened_initial_state = flatten(self.initial_state)

        layer_size = args.block_size * args.num_blocks

        # Scope our new variables to the scope identifier string "rnnlm".
        with tf.variable_scope('rnnlm'):
            # Create new variable softmax_w and softmax_b for output.
            # softmax_w is a weights matrix from the top layer of the model (of size layer_size)
            # to the vocabulary output (of size vocab_size).
            softmax_w = tf.get_variable("softmax_w", [layer_size, args.vocab_size])
            # softmax_b is a bias vector of the ouput characters (of size vocab_size).
            softmax_b = tf.get_variable("softmax_b", [args.vocab_size])
            # Create new variable named 'embedding' to connect the character input to the base layer
            # of the RNN. Its role is the conceptual inverse of softmax_w.
            # It contains the trainable weights from the one-hot input vector to the lowest layer of RNN.
            embedding = tf.get_variable("embedding", [args.vocab_size, layer_size])
            # Create an embedding tensor with tf.nn.embedding_lookup(embedding, self.input_data).
            # This tensor has dimensions batch_size x seq_length x layer_size.
            inputs = tf.nn.embedding_lookup(embedding, self.input_data)

        # TODO: Check arguments parallel_iterations (default uses more memory and less time) and
        # swap_memory (default uses more memory but "minimal (or no) performance penalty")
        outputs, self.final_state = tf.nn.dynamic_rnn(cell, inputs,
                initial_state=self.initial_state, scope='rnnlm')
        # outputs has shape [batch_size, max_time, cell.output_size] because time_major == false.
        # Do we need to transpose the first two dimensions? (Answer: no, this ruins everything.)
        # outputs = tf.transpose(outputs, perm=[1, 0, 2])
        output = tf.reshape(outputs, [-1, layer_size])
        # Obtain logits node by applying output weights and biases to the output tensor.
        # Logits is a tensor of shape [(batch_size * seq_length) x vocab_size].
        # Recall that outputs is a 2D tensor of shape [(batch_size * seq_length) x layer_size],
        # and softmax_w is a 2D tensor of shape [layer_size x vocab_size].
        # The matrix product is therefore a new 2D tensor of [(batch_size * seq_length) x vocab_size].
        # In other words, that multiplication converts a loooong list of layer_size vectors
        # to a loooong list of vocab_size vectors.
        # Then add softmax_b (a single vocab-sized vector) to every row of that list.
        # That gives you the logits!
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        if infer:
            # Convert logits to probabilities. Probs isn't used during training! That node is never calculated.
            # Like logits, probs is a tensor of shape [(batch_size * seq_length) x vocab_size].
            # During sampling, this means it is of shape [1 x vocab_size].
            self.probs = tf.nn.softmax(self.logits)
        else:
            # Create a targets placeholder of shape batch_size x seq_length.
            # Targets will be what output is compared against to calculate loss.
            self.targets = tf.placeholder(tf.int32, [args.batch_size, args.seq_length])
            # seq2seq.sequence_loss_by_example returns 1D float Tensor containing the log-perplexity
            # for each sequence. (Size is batch_size * seq_length.)
            # Targets are reshaped from a [batch_size x seq_length] tensor to a 1D tensor, of the following layout:
            #   target character (batch 0, seq 0)
            #   target character (batch 0, seq 1)
            #   ...
            #   target character (batch 0, seq seq_len-1)
            #   target character (batch 1, seq 0)
            #   ...
            # These targets are compared to the logits to generate loss.
            # Logits: instead of a list of character indices, it's a list of character index probability vectors.
            # seq2seq.sequence_loss_by_example will do the work of generating losses by comparing the one-hot vectors
            # implicitly represented by the target characters against the probability distrutions in logits.
            # It returns a 1D float tensor (a vector) where item i is the log-perplexity of
            # the comparison of the ith logit distribution to the ith one-hot target vector.

            loss = nn_ops.sparse_softmax_cross_entropy_with_logits(
                labels=tf.reshape(self.targets, [-1]), logits=self.logits)

            # Cost is the arithmetic mean of the values of the loss tensor.
            # It is a single-element floating point tensor. This is what the optimizer seeks to minimize.
            self.cost = tf.reduce_mean(loss)
            # Create a tensorboard summary of our cost.
            tf.summary.scalar("cost", self.cost)

            tvars = tf.trainable_variables() # tvars is a python list of all trainable TF Variable objects.
            # tf.gradients returns a list of tensors of length len(tvars) where each tensor is sum(dy/dx).
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),
                     args.grad_clip)
            optimizer = tf.train.AdamOptimizer(self.lr) # Use ADAM optimizer.
            # Zip creates a list of tuples, where each tuple is (variable tensor, gradient tensor).
            # Training op nudges the variables along the gradient, with the given learning rate, using the ADAM optimizer.
            # This is the op that a training session should be instructed to perform.
            self.train_op = optimizer.apply_gradients(zip(grads, tvars))
            #self.train_op = optimizer.minimize(self.cost)
            self.summary_op = tf.summary.merge_all()

    def add_state_to_feed_dict(self, feed_dict, state):
        for i, tensor in enumerate(flatten(state)):
            feed_dict[self._flattened_initial_state[i]] = tensor

    def save_variables_list(self):
        # Return a list of the trainable variables created within the rnnlm model.
        # This consists of the two projection softmax variables (softmax_w and softmax_b),
        # embedding, and all of the weights and biases in the MultiRNNCell model.
        # Save only the trainable variables and the placeholders needed to resume training;
        # discard the rest, including optimizer state.
        save_vars = set(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='rnnlm'))
        save_vars.update({self.lr, self.global_epoch_fraction, self.global_seconds_elapsed})
        return list(save_vars)

    def forward_model(self, sess, state, input_sample):
        '''Run a forward pass. Return the updated hidden state and the output probabilities.'''
        shaped_input = np.array([[input_sample]], np.float32)
        inputs = {self.input_data: shaped_input}
        self.add_state_to_feed_dict(inputs, state)
        [probs, state] = sess.run([self.probs, self.final_state], feed_dict=inputs)
        return probs[0], state

    def trainable_parameter_count(self):
        total_parameters = 0
        for variable in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='rnnlm'):
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        return total_parameters

In [0]:
async def download_file(url, file_name, file_type):
    if file_type == 'exe' or file_name == 'js':
        return
    headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0'
    }
    r = requests.get(url, headers=headers, stream=True)
    with open("images/"+str(file_name)+'.'+str(file_type), 'wb') as f:
    #with open("images/"+str(file_name)+'.jpg', 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

##Makes a directory of images
Yes.This header was nessasary.

In [0]:
!mkdir images

#Main Function
Hecc yus.

In [0]:
client = Bot(description="JADE AI", command_prefix="", pm_help = False)
setup(client)

@client.event
async def on_ready():
	print('Logged in as '+client.user.name+' (ID:'+client.user.id+') | Connected to '+str(len(client.servers))+' servers | Connected to '+
        str(len(set(client.get_all_members())))+' users')
	print('--------')
	print('You are running JadeAI') #Do not change this. This will really help us support you, if you need support.
	return await client.change_presence(game=discord.Game(name="with my " + str(len(set(client.get_all_members()))) + " friends! ||| I've been invited to "+str(len(client.servers)) + " homes, and JD is my prefix!")) #This is buggy, let us know if it doesn't work.


assert sys.version_info >= (3, 3), \
"Must be run in Python 3.3 or later. You are running {}".format(sys.version)
parser = argparse.ArgumentParser()
parser.add_argument('-f', type=str,
                   help='fixes -f')
#parser.add_argument('--save_dir', type=str, default='drive/JadeV3Train/new_save',
parser.add_argument('--save_dir', type=str, default='drive/JadeV3/models/reddit',
                   help='model directory to store checkpointed models')
parser.add_argument('-n', type=int, default=150,
                   help='number of characters to sample')
parser.add_argument('--prime', type=str, default=' ',
                   help='prime text')
parser.add_argument('--beam_width', type=int, default=2,
                   help='Width of the beam for beam search, default 2')
parser.add_argument('--temperature', type=float, default=1.0,
                   help='sampling temperature'
                   '(lower is more conservative, default is 1.0, which is neutral)')
parser.add_argument('--topn', type=int, default=-1,
                    help='at each step, choose from only this many most likely characters;'
                    'set to <0 to disable top-n filtering.')
parser.add_argument('--relevance', type=float, default=-1.,
                   help='amount of "relevance masking/MMI (disabled by default):"'
                   'higher is more pressure, 0.4 is probably as high as it can go without'
                   'noticeably degrading coherence;'
                   'set to <0 to disable relevance masking')
args = parser.parse_args()


def get_paths(input_path):
    if os.path.isfile(input_path):
        # Passed a model rather than a checkpoint directory
        model_path = input_path
        save_dir = os.path.dirname(model_path)
    elif os.path.exists(input_path):
        # Passed a checkpoint directory
        save_dir = input_path
        checkpoint = tf.train.get_checkpoint_state(save_dir)
        if checkpoint:
            model_path = checkpoint.model_checkpoint_path
        else:
            raise ValueError('Checkpoint not found in {}.'.format(save_dir))
    else:
        raise ValueError('save_dir is not a valid path.')
    return model_path, os.path.join(save_dir, 'config.pkl'), os.path.join(save_dir, 'chars_vocab.pkl')

def initial_state(net, sess):
    # Return freshly initialized model states.
    return sess.run(net.zero_state)

def forward_text(net, sess, states, relevance, vocab, prime_text=None):
    if prime_text is not None:
        for char in prime_text:
            if relevance > 0.:
                # Automatically forward the primary net.
                _, states[0] = net.forward_model(sess, states[0], vocab[char])
                # If the token is newline, reset the mask net state; else, forward it.
                if vocab[char] == '\n':
                    states[1] = initial_state(net, sess)
                else:
                    _, states[1] = net.forward_model(sess, states[1], vocab[char])
            else:
                _, states = net.forward_model(sess, states, vocab[char])
    return states

def sanitize_text(vocab, text): # Strip out characters that are not part of the net's vocab.
    return ''.join(i for i in text if i in vocab)

def initial_state_with_relevance_masking(net, sess, relevance):
    if relevance <= 0.: return initial_state(net, sess)
    else: return [initial_state(net, sess), initial_state(net, sess)]

def possibly_escaped_char(raw_chars):
    if raw_chars[-1] == ';':
        for i, c in enumerate(reversed(raw_chars[:-1])):
            if c == ';' or i > 8:
                return raw_chars[-1]
            elif c == '&':
                escape_seq = "".join(raw_chars[-(i + 2):])
                new_seq = html.unescape(escape_seq)
                backspace_seq = "".join(['\b'] * (len(escape_seq)-1))
                diff_length = len(escape_seq) - len(new_seq) - 1
                return backspace_seq + new_seq + "".join([' '] * diff_length) + "".join(['\b'] * diff_length)
    return raw_chars[-1] 

def process_user_command(user_input, states, relevance, temperature, topn, beam_width):
    user_command_entered = False
    reset = False
    try:
        if user_input.startswith('--temperature '):
            user_command_entered = True
            temperature = max(0.001, float(user_input[len('--temperature '):]))
            print("[Temperature set to {}]".format(temperature))
        elif user_input.startswith('--relevance '):
            user_command_entered = True
            new_relevance = float(user_input[len('--relevance '):])
            if relevance <= 0. and new_relevance > 0.:
                states = [states, copy.deepcopy(states)]
            elif relevance > 0. and new_relevance <= 0.:
                states = states[0]
            relevance = new_relevance
            print("[Relevance disabled]" if relevance <= 0. else "[Relevance set to {}]".format(relevance))
        elif user_input.startswith('--topn '):
            user_command_entered = True
            topn = int(user_input[len('--topn '):])
            print("[Top-n filtering disabled]" if topn <= 0 else "[Top-n filtering set to {}]".format(topn))
        elif user_input.startswith('--beam_width '):
            user_command_entered = True
            beam_width = max(1, int(user_input[len('--beam_width '):]))
            print("[Beam width set to {}]".format(beam_width))
        elif user_input.startswith('--reset'):
            user_command_entered = True
            reset = True
            print("[Model state reset]")
    except ValueError:
        print("[Value error with provided argument.]")
    return user_command_entered, reset, states, relevance, temperature, topn, beam_width

def consensus_length(beam_outputs, early_term_token):
    for l in range(len(beam_outputs[0])):
        if l > 0 and beam_outputs[0][l-1] == early_term_token:
            return l-1, True
        for b in beam_outputs[1:]:
            if beam_outputs[0][l] != b[l]: return l, False
    return l, False

def scale_prediction(prediction, temperature):
    if (temperature == 1.0): return prediction # Temperature 1.0 makes no change
    np.seterr(divide='ignore')
    scaled_prediction = np.log(prediction) / temperature
    scaled_prediction = scaled_prediction - np.logaddexp.reduce(scaled_prediction)
    scaled_prediction = np.exp(scaled_prediction)
    np.seterr(divide='warn')
    return scaled_prediction

def forward_with_mask(sess, net, states, input_sample, forward_args):
    # forward_args is a dictionary containing arguments for generating probabilities.
    relevance = forward_args['relevance']
    mask_reset_token = forward_args['mask_reset_token']
    forbidden_token = forward_args['forbidden_token']
    temperature = forward_args['temperature']
    topn = forward_args['topn']

    if relevance <= 0.:
        # No relevance masking.
        prob, states = net.forward_model(sess, states, input_sample)
    else:
        # states should be a 2-length list: [primary net state, mask net state].
        if input_sample == mask_reset_token:
            # Reset the mask probs when reaching mask_reset_token (newline).
            states[1] = initial_state(net, sess)
        primary_prob, states[0] = net.forward_model(sess, states[0], input_sample)
        primary_prob /= sum(primary_prob)
        mask_prob, states[1] = net.forward_model(sess, states[1], input_sample)
        mask_prob /= sum(mask_prob)
        prob = np.exp(np.log(primary_prob) - relevance * np.log(mask_prob))
    # Mask out the forbidden token (">") to prevent the bot from deciding the chat is over)
    prob[forbidden_token] = 0
    # Normalize probabilities so they sum to 1.
    prob = prob / sum(prob)
    # Apply temperature.
    prob = scale_prediction(prob, temperature)
    # Apply top-n filtering if enabled
    if topn > 0:
        prob[np.argsort(prob)[:-topn]] = 0
        prob = prob / sum(prob)
    return prob, states

def beam_search_generator(sess, net, initial_state, initial_sample,
    early_term_token, beam_width, forward_model_fn, forward_args):
    '''Run beam search! Yield consensus tokens sequentially, as a generator;
    return when reaching early_term_token (newline).

    Args:
        sess: tensorflow session reference
        net: tensorflow net graph (must be compatible with the forward_net function)
        initial_state: initial hidden state of the net
        initial_sample: single token (excluding any seed/priming material)
            to start the generation
        early_term_token: stop when the beam reaches consensus on this token
            (but do not return this token).
        beam_width: how many beams to track
        forward_model_fn: function to forward the model, must be of the form:
            probability_output, beam_state =
                    forward_model_fn(sess, net, beam_state, beam_sample, forward_args)
            (Note: probability_output has to be a valid probability distribution!)
        tot_steps: how many tokens to generate before stopping,
            unless already stopped via early_term_token.
    Returns: a generator to yield a sequence of beam-sampled tokens.'''
    # Store state, outputs and probabilities for up to args.beam_width beams.
    # Initialize with just the one starting entry; it will branch to fill the beam
    # in the first step.
    beam_states = [initial_state] # Stores the best activation states
    beam_outputs = [[initial_sample]] # Stores the best generated output sequences so far.
    beam_probs = [1.] # Stores the cumulative normalized probabilities of the beams so far.

    while True:
        # Keep a running list of the best beam branches for next step.
        # Don't actually copy any big data structures yet, just keep references
        # to existing beam state entries, and then clone them as necessary
        # at the end of the generation step.
        new_beam_indices = []
        new_beam_probs = []
        new_beam_samples = []

        # Iterate through the beam entries.
        for beam_index, beam_state in enumerate(beam_states):
            beam_prob = beam_probs[beam_index]
            beam_sample = beam_outputs[beam_index][-1]

            # Forward the model.
            prediction, beam_states[beam_index] = forward_model_fn(
                    sess, net, beam_state, beam_sample, forward_args)

            # Sample best_tokens from the probability distribution.
            # Sample from the scaled probability distribution beam_width choices
            # (but not more than the number of positive probabilities in scaled_prediction).
            count = min(beam_width, sum(1 if p > 0. else 0 for p in prediction))
            best_tokens = np.random.choice(len(prediction), size=count,
                                            replace=False, p=prediction)
            for token in best_tokens:
                prob = prediction[token] * beam_prob
                if len(new_beam_indices) < beam_width:
                    # If we don't have enough new_beam_indices, we automatically qualify.
                    new_beam_indices.append(beam_index)
                    new_beam_probs.append(prob)
                    new_beam_samples.append(token)
                else:
                    # Sample a low-probability beam to possibly replace.
                    np_new_beam_probs = np.array(new_beam_probs)
                    inverse_probs = -np_new_beam_probs + max(np_new_beam_probs) + min(np_new_beam_probs)
                    inverse_probs = inverse_probs / sum(inverse_probs)
                    sampled_beam_index = np.random.choice(beam_width, p=inverse_probs)
                    if new_beam_probs[sampled_beam_index] <= prob:
                        # Replace it.
                        new_beam_indices[sampled_beam_index] = beam_index
                        new_beam_probs[sampled_beam_index] = prob
                        new_beam_samples[sampled_beam_index] = token
        # Replace the old states with the new states, first by referencing and then by copying.
        already_referenced = [False] * beam_width
        new_beam_states = []
        new_beam_outputs = []
        for i, new_index in enumerate(new_beam_indices):
            if already_referenced[new_index]:
                new_beam = copy.deepcopy(beam_states[new_index])
            else:
                new_beam = beam_states[new_index]
                already_referenced[new_index] = True
            new_beam_states.append(new_beam)
            new_beam_outputs.append(beam_outputs[new_index] + [new_beam_samples[i]])
        # Normalize the beam probabilities so they don't drop to zero
        beam_probs = new_beam_probs / sum(new_beam_probs)
        beam_states = new_beam_states
        beam_outputs = new_beam_outputs
        # Prune the agreed portions of the outputs
        # and yield the tokens on which the beam has reached consensus.
        l, early_term = consensus_length(beam_outputs, early_term_token)
        if l > 0:
            for token in beam_outputs[0][:l]: yield token
            beam_outputs = [output[l:] for output in beam_outputs]
        if early_term: return

model_path, config_path, vocab_path = get_paths(args.save_dir)
# Arguments passed to sample.py direct us to a saved model.
# Load the separate arguments by which that model was previously trained.
# That's saved_args. Use those to load the model.
with open(config_path, 'rb') as f:
    saved_args = pickle.load(f)
# Separately load chars and vocab from the save directory.
with open(vocab_path, 'rb') as f:
    chars, vocab = pickle.load(f)
# Create the model from the saved arguments, in inference mode.
print("Creating model...")
saved_args.batch_size = args.beam_width
net = Model(saved_args, True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# Make tensorflow less verbose; filter out info (1+) and warnings (2+) but not errors (3).
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
sess = tf.Session(config=config)
#tf.global_variables_initializer().run()
saver = tf.train.Saver(net.save_variables_list())
# Restore the saved variables, replacing the initialized values.
print("Restoring weights...")
saver.restore(sess, model_path)
states = initial_state_with_relevance_masking(net, sess, args.relevance)

@client.event
async def on_message(message):
  global states
  if not message.server == None and not str(message.channel.id) == "471788266052386816" and not message.author.bot:
    if message.content.startswith('JD ') or message.content.startswith('jd '):
      file = open("drive/JadeV3/logs/log.txt","a") 
      ModMessage = message.content[3:]
      t1 = time.perf_counter()
      print('\n' + str(message.server)+ ", " + str(message.channel.id) + " ::: " + datetime.date.today().strftime("%A") + ", " + datetime.date.today().strftime("%B") + " "+
            datetime.date.today().strftime("%d") + '\n' + str(message.author) + ": " + ModMessage)
      file.write('\n' + str(message.server) + " ::: " + datetime.date.today().strftime("%A") + ", " + datetime.date.today().strftime("%B") + " "+
                 datetime.date.today().strftime("%d") + '\n' + str(message.author) + ": " + ModMessage)
      out_txt = ""
      user_input = ModMessage
      user_command_entered, reset, states, relevance, temperature, topn, beam_width = process_user_command(
          user_input, states, args.relevance, args.temperature, args.topn, args.beam_width)
      if reset: states = initial_state_with_relevance_masking(net, sess, args.relevance)
      if not user_command_entered:
          states = forward_text(net, sess, states, args.relevance, vocab, sanitize_text(vocab, "> " + user_input + "\n>"))
          computer_response_generator = beam_search_generator(sess=sess, net=net,
              initial_state=copy.deepcopy(states), initial_sample=vocab[' '],
              early_term_token=vocab['\n'], beam_width=args.beam_width, forward_model_fn=forward_with_mask,
              forward_args={'relevance':args.relevance, 'mask_reset_token':vocab['\n'], 'forbidden_token':vocab['>'],
                              'temperature':args.temperature, 'topn':args.topn})
          out_chars = []
          await client.send_typing(message.channel)
          for i, char_token in enumerate(computer_response_generator):
              out_chars.append(chars[char_token])
              out_txt = out_txt + possibly_escaped_char(out_chars)
              #print(possibly_escaped_char(out_chars), end='', flush=True)
              states = forward_text(net, sess, states, relevance, vocab, chars[char_token])
              if i >= args.n: break
          t2 = time.perf_counter()
          if user_input.lower() == "ping":
            out_txt = "pseudo-ping: " + str((round((t2-t1)*1000))) + "ms"
          print(out_txt)
          file.write("\n" + out_txt)
          await client.send_message(message.channel,out_txt)
          states = forward_text(net, sess, states, relevance, vocab, sanitize_text(vocab, "\n> "))
          file.close() 
          
    if message.content.startswith('JI') or message.content.startswith('ji'):
      #vote= await dblpy.get_upvote_info()
      #print(vote)
      ModMessage = message.content[2:]
      t1 = time.perf_counter()
      #if not ModMessage == "":
        #print("")
        #try:
        #  !wget ModMessage -O images/custom_img_1.jpg
        #  !ls ./images/
        #except Exception as e:
        #  await client.send_message(message.channel,e)
      #else:
      await client.send_typing(message.channel)
      if ModMessage == "":
        try:
          x=(str(message.attachments[0]))
          y = x.split(" ")
          z = y[3].strip("'")
          z = z.strip(",")
          z = z.strip("'")
          print("\n"+ str(message.author) + ": " + z)
          await download_file(z,'custom_img_1',"jpg")
        except Exception as e:
          await client.send_message(message.channel,e)
      else:
        ModMessage = message.content[3:]
        print("\n"+str(message.author) + ": " + ModMessage)
        await download_file(ModMessage,'custom_img_1',"jpg")
        
                
      await client.send_typing(message.channel)
      PATH_TO_TEST_IMAGES_DIR = 'images'
      TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'custom_img_1.jpg') ]
      IMAGE_SIZE = (12, 8)
      
      with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess1:
          image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
          detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
          detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
          detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
          num_detections = detection_graph.get_tensor_by_name('num_detections:0')
          await client.send_typing(message.channel)
          for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess1.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            await client.send_typing(message.channel)
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=4)
            await client.send_typing(message.channel)
            plt.figure(figsize=IMAGE_SIZE)
            #plt.imshow(image_np)
            scipy.misc.imsave('images/result.jpg', image_np)
            t2 = time.perf_counter()
            await client.send_file(message.channel,'images/result.jpg')
            await client.send_message(message.channel, ("Completed in: " + str((round((t2-t1)*1000))) + "ms"))
            
  if message.content.startswith('JT') or message.content.startswith('jt'):
      #vote= await dblpy.get_upvote_info()
      #print(vote)
      ModMessage = message.content[2:]
      t1 = time.perf_counter()
      #if not ModMessage == "":
        #print("")
        #try:
        #  !wget ModMessage -O images/custom_img_1.jpg
        #  !ls ./images/
        #except Exception as e:
        #  await client.send_message(message.channel,e)
      #else:
      await client.send_typing(message.channel)
      if ModMessage == "":
        try:
          x=(str(message.attachments[0]))
          y = x.split(" ")
          z = y[3].strip("'")
          z = z.strip(",")
          z = z.strip("'")
          print("\n"+str(message.author) + ": " + z)
          await download_file(z,'custom_img_1',"jpg")
        except Exception as e:
          await client.send_message(message.channel,e)
      else:
        ModMessage = message.content[3:]
        print("\n"+str(message.author) + ": " + ModMessage)
        await download_file(ModMessage,'custom_img_1',"jpg")
      
      try:
        img = cv2.imread('images/custom_img_1.jpg')
      except:
        img = cv2.imread('images/custom_img_1.png')
      
      s=(pytesseract.image_to_string(img))
      print(s)
      #v=(pytesseract.image_to_boxes(img))
      #print(v)
      try:
        await client.send_message(message.channel, s)
      except:
        await client.send_message(message.channel, "I couldn't find anythinng readable...")
      #await client.send_message(message.channel, v)
      
  if message.content.startswith('JS') or message.content.startswith('js'):
      ModMessage = message.content[2:]
      t1 = time.perf_counter()
      #if not ModMessage == "":
        #print("")
        #try:
        #  !wget ModMessage -O images/custom_img_1.jpg
        #  !ls ./images/
        #except Exception as e:
        #  await client.send_message(message.channel,e)
      #else:
      await client.send_typing(message.channel)
      if ModMessage == "":
        try:
          x=(str(message.attachments[0]))
          y = x.split(" ")
          z = y[3].strip("'")
          z = z.strip(",")
          z = z.strip("'")
          print("\n"+str(message.author) + ": " + z)
          await download_file(z,'custom_img_1',"jpg")
        except Exception as e:
          await client.send_message(message.channel,e)
        
        try:
          x=(str(message.attachments[1]))
          y = x.split(" ")
          z = y[3].strip("'")
          z = z.strip(",")
          z = z.strip("'")
          print(message.author + ": " + z)
          await download_file(z,'custom_img_2',"jpg")
        except Exception as e:
          await client.send_message(message.channel,e)
      else:
        ModMessage = message.content[3:]
        p=ModMessage.split(", ")
        for item in p:
          print("\n"+str(message.author) + ": " + item)
        await download_file(p[0],'custom_img_2',"jpg")
        await download_file(p[1],'custom_img_1',"jpg")
        
      
      content_file_name = 'custom_img_1.jpg'
      style_file_name = 'custom_img_2.jpg'
      
      content_weight = 5e0
      style_weight = 1e4
      tv_weight = 1e3
      learning_rate = 1e0
      iterations =  100
      checkpoint_iterations = 1000
      print_iterations = 25
        
      t1 = time.perf_counter()
      await client.send_typing(message.channel)
      image_content = scipy.misc.imread('./images/'+content_file_name)
      image_content = image_content.astype('float32')
      image_content = np.ndarray.reshape(image_content,((1,) + image_content.shape)) # 1 means batch_size
      #image_content = np.ndarray.reshape(image_content.shape + (1,)) it is needed to handle gray scale image 

      image_style = scipy.misc.imread('./images/'+style_file_name)
      image_style = image_style.astype('float32')
      image_style = np.ndarray.reshape(image_style,((1,) + image_style.shape)) # 1 means batch_size
      #image_style = np.ndarray.reshape(image_style.shape + (1,)) it is needed to handle gray scale image

      mean = data['normalization'][0][0][0]
      mean_pixel = np.mean(mean, axis=(0, 1))

      # CONTENT_LAYER = 'relu4_2'# CONTE 
      # content_features = {}

      # with tf.Session() as sess:
      #     content_pre = preprocess(image_content, mean_pixel)
      #     content_net = net(np.squeeze(data['layers']), content_pre)
      #     content_features[CONTENT_LAYER] = content_net[CONTENT_LAYER].eval()

      CONTENT_LAYERS = ('conv1_1', 'conv2_1', 'conv4_1', 'conv4_2')
      content_features = {}

      with tf.Session() as sess2:
          content_pre = preprocess(image_content, mean_pixel)
          content_net = netp(content_pre)
          for layer in CONTENT_LAYERS:
              content_features[layer] = content_net[layer].eval()

      STYLE_LAYERS = ('conv3_1','conv5_1')
      style_features = {}

      await client.send_typing(message.channel)
      with tf.Session() as sess3:
          style_pre = preprocess(image_style, mean_pixel)
          style_net = netp(style_pre)
          for layer in STYLE_LAYERS:
              features = style_net[layer].eval()
              features = np.reshape(features, (-1, features.shape[3]))
              gram = np.matmul(features.T, features) / features.size
              style_features[layer] = gram

      # make stylized image using backpropogation
      initial = None
      #initial = scipy.misc.imread('./images/cat.jpg')
      if initial is None:
          noise = np.random.normal(size=image_content.shape, scale=np.std(image_content) * 0.1)
          initial = tf.random_normal(image_content.shape) * 0.256
      else:
          initial = np.array([preprocess(initial, mean_pixel)])
          initial = initial.astype('float32')

      image = tf.Variable(initial)
      image_net = netp(image)

      # content loss
      # content_loss = content_weight * (2 * tf.nn.l2_loss(
      #         image_net[CONTENT_LAYER] - content_features[CONTENT_LAYER]) / 
      #         content_features[CONTENT_LAYER].size)
      content_loss = 0
      content_losses = []
      for content_layer in CONTENT_LAYERS:
          content_losses.append(2 * tf.nn.l2_loss(
                               image_net[content_layer] - content_features[content_layer]) / 
                               content_features[content_layer].size)
      content_loss += content_weight * reduce(tf.add, content_losses)

      # style loss
      style_loss = 0
      style_losses = []
      for style_layer in STYLE_LAYERS:
          layer = image_net[style_layer]
          _, height, width, number = map(lambda i: i.value, layer.get_shape())
          size = height * width * number
          feats = tf.reshape(layer, (-1, number))
          gram = tf.matmul(tf.transpose(feats), feats) / size
          style_gram = style_features[style_layer]
          style_losses.append(2 * tf.nn.l2_loss(gram - style_gram) / style_gram.size)
      style_loss += style_weight * reduce(tf.add, style_losses)

      await client.send_typing(message.channel)
      # total variation denoising
      tv_y_size = _tensor_size(image[:,1:,:,:])
      tv_x_size = _tensor_size(image[:,:,1:,:])
      tv_loss = tv_weight * 2 * (
        (tf.nn.l2_loss(image[:,1:,:,:] - image[:,:image_content.shape[1]-1,:,:]) /
            tv_y_size) +
        (tf.nn.l2_loss(image[:,:,1:,:] - image[:,:,:image_content.shape[2]-1,:]) /
            tv_x_size))

      # overall loss# overal 
      loss = content_loss + style_loss + tv_loss

      # optimizer setup
      train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

      # optimization
      best_loss = float('inf')
      best = None

      with tf.Session() as sess4:
          sess4.run(tf.global_variables_initializer())
          for i in range(iterations):
              await client.send_typing(message.channel)
              train_step.run()
        
              if i % checkpoint_iterations == 0 or i == iterations - 1:
                this_loss = loss.eval()
                if this_loss < best_loss:
                  best_loss = this_loss
                  best = image.eval()
                  # save a check point
                  try:
                      os.makedirs('./checks/'+str.split(content_file_name,'.')[0])
                  except OSError:
                      pass
                  timestr = time.strftime("%Y%m%d_%H%M%S")
                  filename_cp = './checks/'+str.split(content_file_name,'.')[0]+'/'+timestr+'.jpg'
                  cp = unprocess(best.reshape(image_content.shape[1:]), mean_pixel)
                  #imsave("result.jpg", cp)
        
              if i % print_iterations == 0 or i == iterations - 1:
                  print('Iteration %d/%d' % (i + 1, iterations))
                  #print('  content loss: %g' % content_loss.eval())
                  #print('    style loss: %g' % style_loss.eval())
                  #print('       tv loss: %g' % tv_loss.eval())
                  print('    total loss: %g' % loss.eval())

          output = unprocess(best.reshape(image_content.shape[1:]), mean_pixel)
          imsave('images/result.jpg', output)
          
      t2 = time.perf_counter()
      await client.send_file(message.channel,'images/result.jpg')
      await client.send_message(message.channel, ("Completed in: " + str((round((t2-t1)*1000))) + "ms"))
      
  elif message.server == None and not message.author.bot:
    file = open("drive/JadeV3/logs/log.txt","a")
    print('\n' + str(message.server) + " ::: " + datetime.date.today().strftime("%A") + ", " + datetime.date.today().strftime("%B") + " "+
          datetime.date.today().strftime("%d") + '\n' + str(message.author) + ": " + message.content)
    file.write('\n' + str(message.server) + " ::: " + datetime.date.today().strftime("%A") + ", " + datetime.date.today().strftime("%B") + " "+
               datetime.date.today().strftime("%d") + '\n' + str(message.author) + ": " + message.content)
    out_txt = ""
    user_input = message.content
    user_command_entered, reset, states, relevance, temperature, topn, beam_width = process_user_command(
        user_input, states, args.relevance, args.temperature, args.topn, args.beam_width)
    if reset: states = initial_state_with_relevance_masking(net, sess, args.relevance)
    if not user_command_entered:
        states = forward_text(net, sess, states, args.relevance, vocab, sanitize_text(vocab, "> " + user_input + "\n>"))
        computer_response_generator = beam_search_generator(sess=sess, net=net,
            initial_state=copy.deepcopy(states), initial_sample=vocab[' '],
            early_term_token=vocab['\n'], beam_width=args.beam_width, forward_model_fn=forward_with_mask,
            forward_args={'relevance':args.relevance, 'mask_reset_token':vocab['\n'], 'forbidden_token':vocab['>'],
                            'temperature':args.temperature, 'topn':args.topn})
        out_chars = []
        await client.send_typing(message.channel)
        for i, char_token in enumerate(computer_response_generator):
            out_chars.append(chars[char_token])
            out_txt = out_txt + possibly_escaped_char(out_chars)
            #print(possibly_escaped_char(out_chars), end='', flush=True)
            states = forward_text(net, sess, states, relevance, vocab, chars[char_token])
            if i >= args.n: break
        print(out_txt)
        file.write("\n" + out_txt)
        await client.send_message(message.channel,out_txt)
        states = forward_text(net, sess, states, relevance, vocab, sanitize_text(vocab, "\n> "))
        file.close() 
        
  elif not message.server == None and str(message.channel.id) == "471788266052386816" and not message.author.bot:
    file = open("drive/JadeV3/logs/log.txt","a")
    print('\n' + str(message.server) + " ::: " + datetime.date.today().strftime("%A") + ", " + datetime.date.today().strftime("%B") + " "+
          datetime.date.today().strftime("%d") + '\n' + str(message.author) + ": " + message.content)
    file.write('\n' + str(message.server) + " ::: " + datetime.date.today().strftime("%A") + ", " + datetime.date.today().strftime("%B") + " "+
               datetime.date.today().strftime("%d") + '\n' + str(message.author) + ": " + message.content)
    out_txt = ""
    user_input = message.content
    user_command_entered, reset, states, relevance, temperature, topn, beam_width = process_user_command(
        user_input, states, args.relevance, args.temperature, args.topn, args.beam_width)
    if reset: states = initial_state_with_relevance_masking(net, sess, args.relevance)
    if not user_command_entered:
        states = forward_text(net, sess, states, args.relevance, vocab, sanitize_text(vocab, "> " + user_input + "\n>"))
        computer_response_generator = beam_search_generator(sess=sess, net=net,
            initial_state=copy.deepcopy(states), initial_sample=vocab[' '],
            early_term_token=vocab['\n'], beam_width=args.beam_width, forward_model_fn=forward_with_mask,
            forward_args={'relevance':args.relevance, 'mask_reset_token':vocab['\n'], 'forbidden_token':vocab['>'],
                            'temperature':args.temperature, 'topn':args.topn})
        out_chars = []
        await client.send_typing(message.channel)
        for i, char_token in enumerate(computer_response_generator):
            out_chars.append(chars[char_token])
            out_txt = out_txt + possibly_escaped_char(out_chars)
            #print(possibly_escaped_char(out_chars), end='', flush=True)
            states = forward_text(net, sess, states, relevance, vocab, chars[char_token])
            if i >= args.n: break
        print(out_txt)
        file.write("\n" + out_txt)
        await client.send_message(message.channel,out_txt)
        states = forward_text(net, sess, states, relevance, vocab, sanitize_text(vocab, "\n> "))
        file.close() 
  try:
    await client.change_presence(game=discord.Game(name="with my " + str(len(set(client.get_all_members()))) + " friends! ||| I've been invited to "+str(len(client.servers)) + " homes, and JD is my prefix!"))
  except:
    return
      
client.run('Token')